In [3]:
import ee
import geemap

ee.Initialize()

In [25]:
# Import AIM R&W Polygons 
rw = ee.FeatureCollection('projects/dri-apps/assets/blm-riparian/aim-rw-footprints-v1')

# Import NDVI and add a date and scale by 0.001
def add_date_and_scale(img):
        date = ee.Date(img.get('system:time_start')).advance(15, 'days').format('YYYYMMdd');
        return img.multiply(0.001).set({'Date': ee.String(date), 'system:time_start': img.get('system:time_start')})

# Filter dates and apply scaling function
ndvi_coll = ee.ImageCollection("projects/rap-data-365417/assets/ndvi-composites-v1-conus")\
        .filterDate('2018-01-01', '2024-12-31')\
        .map(add_date_and_scale)

In [27]:
# Initialize a geemap object
Map = geemap.Map()

# Visualize NDVI and AIM R&W plots in the map
# Set visualization for gee map object
ndvi_mean = ndvi_coll.filterDate('2024-06-01', '2024-08-31').mean()
Map.addLayer(ndvi_mean, {'min': 0, 'max': 0.7}, 'NDVI Mean in summer 2024')
Map.addLayer(rw, {}, 'AIM R&W Polygons')

# Display the map
Map.centerObject(rw, 6)
Map

Map(center=[41.51825196949538, -119.51349739333641], controls=(WidgetControl(options=['position', 'transparent…

In [30]:
# Convert NDVI to image to extract timeseries and rename bands by date
ndvi_img = ndvi_coll.toBands().rename(ndvi_coll.aggregate_array('Date'))

# Extract NDVI timeseries for AIM R&W Polygons
ndvi_rr = ndvi_img.reduceRegions(collection = rw, reducer = ee.Reducer.mean(), scale = 30, crs = 'EPSG:4326')

# Export data to Google Drive
task = ee.batch.Export.table.toDrive(collection = ndvi_rr, description = 'aim_rw_ndvi', fileFormat = 'CSV')
task.start()